In [1]:
; cl-waffeを読み込む
(load "../../cl-waffe.asd")
(ql:quickload :cl-waffe :silent t)
(use-package :cl-waffe)

T

(:CL-WAFFE)

T

Unable to find framework CUDA


In [2]:
#+sbcl(declaim (sb-ext:muffle-conditions cl:simple-warning))

((#<SB-KERNEL::CONDITION-CLASSOID SIMPLE-WARNING> . MUFFLE-WARNING))

SB-INT:SIMPLE-STYLE-WARNING: DECLAIM where DECLARE was probably intended


### 行列の初期化
関数`(!randn dim)`は平均0分散1の標準分布をサンプリングし、与えられた次元数`dim`の行列を返します。

In [3]:
(!randn `(10 10))

#Const(((-0.54... 0.253... ~ -0.35... -2.36...)        
                 ...
        (-1.44... -1.51... ~ -0.72... 0.676...)) :mgl t :shape (10 10) :backward NIL)

### 変数を定義する。
上記の関数で生成された行列は、Const(定数)と見做され、逆伝播時に勾配を生成しません。

マクロ`(parameter tensor)`を介して受け取った定数を変数にします。

parameterマクロの前後で計算ノードは途切れることに注意してください。

In [4]:
(parameter *)

#Parameter{((-0.54... 0.253... ~ -0.35... -2.36...)            
                         ...
            (-1.44... -1.51... ~ -0.72... 0.676...)) :mgl t :shape (10 10) :backward NIL}

### 計算ノードを構築する

マクロ`(with-no-grad &body body)`内部でない限り、cl-waffeのノードを介した計算は計算ノードを構築します。

計算ノードが構築されていたら、TensorをPrintした時に`:backward ノード名`と表示されます。

In [5]:
(defparameter a (!randn `(3 3)))

(let ((result (!add a 0.0)))
     (print result)
     (print (cl-waffe::waffetensor-state result)))
nil

A

<Node: ADDTENSOR{W918}>

NIL


#Const(((0.625... -1.29... 2.044...)        
                 ...
        (0.575... -1.00... -0.62...)) :mgl t :shape (3 3) :backward <Node: ADDTENSOR{W918}>) 
<Node: ADDTENSOR{W918}> 

## 順伝播と逆伝播

線形回帰モデル
$$
y=Ax+b
$$
を計算する。

構築された計算ノードは、最後のノードを`(backward out)`のように呼び出すことで逆伝播される。

逆伝播はスカラー値に対してでないと定義されないため、最後の計算ノードは損失関数や、`!sum`, `!mean`等になる。

各変数は`(grad tensor)`で勾配を取り出される。（勾配はTensorのデータ構造に依る、構造体`WaffeTensor`ではないことに注意）

In [6]:
(defparameter weight (parameter (!randn `(3 3))))
(defparameter x      (parameter (!randn `(3 3))))

(defparameter bias   (parameter (!randn `(1 3))))

(let ((out (!add (!matmul weight x) bias)))
     (print out)
     
     (setq out (!sum out))
     
     (backward out)
     
     (print (grad weight))
     (print (grad x))
     (print (grad bias)))

WEIGHT

X

BIAS

#<MGL-MAT:MAT 1x3 AF #2A((0.11111112 0.11111112 0.11111112))>


#Const(((0.005... 0.664... 2.546...)        
                 ...
        (-0.42... 0.970... 1.887...)) :mgl t :shape (3 3) :backward <Node: BROADCASTINGADDTENSOR{W943}>) 
#<MGL-MAT:MAT 3x3 F #2A((-0.117892794 -0.08790654 0.1289539)
                        (-0.117892794 -0.08790654 0.1289539)
                        (-0.117892794 -0.08790654 0.1289539))> 
#<MGL-MAT:MAT 3x3 F #2A((0.15121937 0.15121937 0.15121937)
                        (0.026950078 0.026950078 0.026950078)
                        (-0.2382441 -0.2382441 -0.2382441))> 
#<MGL-MAT:MAT 1x3 F #2A((0.11111112 0.11111112 0.11111112))> 

## ノードを定義する

[defnode](https://hikettei.github.io/cl-waffe-docs/docs/cl-waffe.html#2-defnode)マクロを用いて計算ノードの順伝播と逆伝播を定義できます。

引用元：[僕のQiitaの記事](https://qiita.com/hikettei/items/f38e0bba89795ec8bff9#%E8%87%AA%E5%8B%95%E5%BE%AE%E5%88%86)に置いてあります。

## 自動微分

Readmeの英語では`Automatic Differentiation`と記述していますが、`Automatic Backpropagation`の方が適切だと思います。回帰や判別等、教師あり学習や深層学習でモデルを最適化する際に利用する勾配を計算します。(自動微分については`(defnode)`の説明に必要な部分しか書かないので詳しく知りたい方は各自調べてください。[自動微分](https://www.google.com/search?q=%E8%87%AA%E5%8B%95%E5%BE%AE%E5%88%86))

勾配は、順伝播時に構築された計算ノードの:backwardスロットをChain Ruleに従って探索することで計算できます。大体のフレームワークと同じくトップダウン型自動微分で、機械学習なら損失関数->各パラメーターみたいな順番でノードを探索します。

通常ニューラルネットワークではパラメーターは行列、入力はベクトルですが、ここでは簡単のため全てスカラーとします。

$$
X, w\in\mathbb{R}
$$

はそれぞれ入力と学習するパラメーターとします

説明のための仮のネットワークが

$$
f(x) = wx
$$

を用いて

$$
out_1 = f(X)
$$

$$
out_2 = sin(out_1)
$$

$$
out_3 = cos(out_2)
$$

という3層の構造から成り立ち、ネットワーク全体は

$$
out_3 = cos(sin(f(X)))
$$

と表せるとしましょう。

out_3の値から上の式の微分を示します。

まず、`out_3=cos(out_2)`の微分(劣微分)は合成関数の微分より

$$
\frac{\partial{cos(out_2)}}{\partial{out_2}} = -sin(out_2)\cdot{\frac{\partial{sin(out_1)}}{\partial{out_1}}}
$$

同様に、`out_1=sin(f(x))`は

$$
{\frac{\partial{sin(out_1)}}{\partial{out_1}}}=f(out_1)\cdot{\frac{\partial{f(X)}}{\partial{X}}}
$$

また、f(x)の微分は

$$
{\frac{\partial{f(X)}}{\partial{X}}}=w
$$

ですのでXの勾配が求まりました。上の式でwについて偏微分していたら

$$
{\frac{\partial{f(X)}}{\partial{w}}}=X
$$

となりwの勾配が求まります。

上記の例は各ノードが自身の出力と上位の関数の勾配のみを用いて自身の勾配を求め、それを下位のノードに伝播していくことで機械的に勾配の計算ができることを示しています。これは単純な計算木での例ですが、ニューラルネットワークのような複雑な計算木が構築されてもこのルールに従うことで機械的に勾配の計算ができることは伝わったでしょうか。

## defnodeで表現する

cl-waffeでは`cos` `sin`がそれぞれノードに相当し、`(defnode)`を通じて順伝播と逆伝播を定義します。

前の項で述べた数式をcl-waffeを用いてCommon Lispのプログラムで表現するには、まず`sin` `cos` `f`関数をノードとして定義する必要があります。`(defnode)`マクロを用いて


In [7]:
;sin(x)
(defnode SinNode ()
  :parameters ((xi nil))
  :forward ((x)
            (save-for-backward xi x)
            (call-and-dispatch-kernel :sin nil nil x))
  :backward ((dy)
             (list (!mul dy (!cos (self xi))))))

;cos(x)
(defnode CosNode ()
  :parameters ((xi nil))
  :forward ((x)
            (save-for-backward xi x)
            (call-and-dispatch-kernel :cos nil nil x))
  :backward ((dy)
             (list (!mul dy (!mul -1.0 (!sin (self xi)))))))

;f(x)
(defmodel F ()
  :parameters ((w (parameter (const 1.0))))
  :forward ((x)
            (!mul (self w) x)))

NIL

NIL

NIL

SB-INT:SIMPLE-STYLE-WARNING: Cannot
                                                                                          find
                                                                                          type
                                                                                          for
                                                                                          specializer
                                                                                          COMMON-LISP-USER::SINNODE
                                                                                          when
                                                                                          executing
                                                                                          SB-PCL:SPECIALIZER-TYPE-SPECIFIER
                                                                                          for
                                                 


(補足: `:parameters`はノードが順伝播時と逆伝播時で共有する変数の一覧で`(self パラメーター名)`でアクセス可能かつsetfable, `save-for-backward`は逆伝播が呼び出されうる状況のみおいて変数を自身のパラメーターに保存するマクロ, `call-and-dispatch-kernel`は引数の型に応じて適切なcl-waffeのカーネルを呼び出す関数です。マクロ`(parameter tensor)`は受け取ったWaffeTensorを学習可能パラメーターにします。)

四則演算など複数の変数を引数にする場合は計算ノードの分岐として表現します。ですから:backwardは必ずlistで返す必要があり、たとえばforwardの引数なら(x y)なら返り値は(xの勾配 yの勾配)のように記述してください。

関数fは単純にwとxの積ですから、ノードを自分で定義しなくても`(defmodel)`と自動微分を用いて定義します。(もちろんsinやcosのノードは標準で定義されています。)

## 実際に計算してみる

実際に上記の計算を行ってみましょう。

In [8]:
; x をパラメーター（最適化するパラメーター）として初期化
(setq x (parameter (const 1.0)))

#Parameter{1.0}

In [9]:
(setq f-model (F)) ; 後で使うので変数に保存
; [Model: F :ident {W2142}]

(setq out_1 (call f-model x))
;#Const(1.0)
(setq out_2 (call (CosNode) out_1))
;#Const(0.5403023)
(setq out_3 (call (SinNode) out_2))
;#Const(0.51439524)

[Model: F :ident {W1054}]

#Const(1.0)

#Const(0.5403023)

#Const(0.51439524)

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::F-MODEL
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::F-MODEL
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::OUT_1
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::OUT_1
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::OUT_2
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::OUT_2
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::OUT_3


逆伝播は`(backward out)`という関数で行えます。outはスカラー値でWaffeTensorである必要があります。`(with-verbose)`マクロを用いて計算ノードを可視化しながら逆伝播してみましょう。

In [10]:
(with-verbose
    (backward out_3))

NIL

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::OUT_3
<Node: SINNODE{W1111}>
<Node: COSNODE{W1093}>
 <Node: MULTENSOR{W1082}>
  <The End of Node>
  <The End of Node>


## モデルを使う

`defmodel`マクロを介してモデルを定義できます。

モデルの内部では基本的に自動微分を用います。

モデル内部に保存されたTensorの中で`(parameter tensor)`を用いて定義されたものは学習可能パラメーターとみなされ、最適化関数を呼び出したときに破壊的に変更され最適化されます。


`cl-waffe.nn:linear`関数を用いて、線形回帰を行うモデルを定義してみましょう。

In [11]:
(defmodel MyLinearLayer (in-features out-features &optional (bias T))
    :parameters ((weight
                  (parameter (!mul 0.01 (!randn `(,in-features ,out-features))))
                  :type waffetensor)
                 (bias (if bias
                           (parameter (!zeros `(1 ,out-features)))
                           nil)))
    :forward ((x)
              (cl-waffe.nn:linear x (self weight) (self bias))))

NIL

SB-INT:SIMPLE-STYLE-WARNING: Cannot find type for specializer
                             COMMON-LISP-USER::MYLINEARLAYER when executing
                             SB-PCL:SPECIALIZER-TYPE-SPECIFIER for a
                             STANDARD-METHOD of a STANDARD-GENERIC-FUNCTION.
SB-INT:SIMPLE-STYLE-WARNING: Cannot find type for specializer
                             COMMON-LISP-USER::MYLINEARLAYER when executing
                             SB-PCL:SPECIALIZER-TYPE-SPECIFIER for a
                             STANDARD-METHOD of a STANDARD-GENERIC-FUNCTION.
SB-INT:SIMPLE-STYLE-WARNING: Cannot find type for specializer
                             COMMON-LISP-USER::MYLINEARLAYER when executing
                             SB-PCL:SPECIALIZER-TYPE-SPECIFIER for a
                             STANDARD-METHOD of a STANDARD-GENERIC-FUNCTION.
SB-INT:SIMPLE-STYLE-WARNING: undefined type: COMMON-LISP-USER::MYLINEARLAYER


In [14]:
(defparameter *my-model* (MyLinearLayer 100 10))
(call *my-model* (!randn `(100 100)))

*MY-MODEL*

#Const(((0.086... 0.071... ~ 0.131... 0.074...)        
                 ...
        (-0.08... 0.023... ~ 0.003... -0.02...)) :mgl t :shape (100 10) :backward <Node: BROADCASTINGADDTENSOR{W1267}>)

In [15]:
(!sum *)
(backward *)

#Const(-1.0428165)

NIL

In [19]:
(const (grad (mylinearlayer-weight *my-model*))) ; (grad)で勾配を取り出すと短くPrintされないので、WaffeTensorにしている。

#Const(((-0.00... -0.00... ~ -0.00... -0.00...)        
                 ...
        (-0.00... -0.00... ~ -0.00... -0.00...)) :mgl t :shape (100 10) :backward NIL)